# SF3D API Server - Complete Installation (v5)

**COMPLETE SOLUTION**: Installs all SF3D dependencies correctly.

## Steps:
1. Run Cell 1: Clone SF3D and install ALL dependencies
2. **RESTART KERNEL**
3. Run Cell 2: Load model
4. Run Cell 3: Define API
5. Run Cell 4: Start server

## Cell 1: Complete Installation

**Clones SF3D and installs from its requirements.txt**

In [ ]:
%%bash
set -e  # Exit on any error

# Setup paths
USER=${USER:-user}
WORK_DIR="/tmp/sf3d_${USER}"
VENV_DIR="$WORK_DIR/venv"
SF3D_DIR="$WORK_DIR/stable-fast-3d"
CACHE_DIR="$WORK_DIR/cache"

echo "Working in: $WORK_DIR"
echo ""

# Create directories
mkdir -p "$WORK_DIR" "$CACHE_DIR"

# Clone SF3D first (to get requirements.txt)
if [ ! -d "$SF3D_DIR" ]; then
    echo "Cloning SF3D repository..."
    git clone https://github.com/Stability-AI/stable-fast-3d.git "$SF3D_DIR"
    echo "✅ SF3D cloned"
else
    echo "✅ SF3D already cloned"
fi

echo ""

# Create venv
if [ ! -d "$VENV_DIR" ]; then
    echo "Creating virtual environment..."
    python -m venv "$VENV_DIR"
    echo "✅ Venv created"
else
    echo "✅ Venv exists"
fi

# Activate venv
source "$VENV_DIR/bin/activate"

# Set cache locations
export PIP_CACHE_DIR="$CACHE_DIR/pip"
export HF_HOME="$CACHE_DIR/huggingface"

echo ""
echo "Installing dependencies (this will take 5-10 minutes)..."
echo ""

# Upgrade pip
pip install --upgrade pip setuptools wheel

# Install PyTorch FIRST with CUDA 11.8
echo "Installing PyTorch with CUDA 11.8..."
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install from SF3D requirements if it exists
if [ -f "$SF3D_DIR/requirements.txt" ]; then
    echo ""
    echo "Installing from SF3D requirements.txt..."
    pip install -r "$SF3D_DIR/requirements.txt"
fi

# Install additional known dependencies
echo ""
echo "Installing additional dependencies..."
pip install \
    fastapi \
    uvicorn[standard] \
    python-multipart \
    pillow \
    numpy \
    omegaconf \
    einops \
    trimesh \
    pymeshlab \
    transformers \
    accelerate \
    safetensors \
    huggingface-hub \
    opencv-python \
    imageio \
    rembg \
    jaxtyping \
    diffusers \
    gpytoolbox \
    kiui

# Try to install xformers (may fail, but optional)
echo ""
echo "Attempting xformers installation (may take time or fail - it's optional)..."
pip install xformers || echo "⚠️  xformers install failed (this is OK, it's optional)"

# Install pynanoinstantmeshes (may need special handling)
echo ""
echo "Installing pynanoinstantmeshes..."
pip install git+https://github.com/ashawkey/pynanoinstantmeshes.git || \
pip install pynanoinstantmeshes || \
echo "⚠️  pynanoinstantmeshes install failed (may cause issues)"

echo ""
echo "✅ Installation complete!"
echo ""
echo "Installed packages:"
pip list | grep -E '(torch|sf3d|fastapi|jaxtyping|gpytoolbox|trimesh)' || true
echo ""
echo "⚠️  CRITICAL: Now go to Kernel → Restart Kernel"
echo "   Then run Cell 2 to load the model."

## Cell 2: Load SF3D Model

**AFTER KERNEL RESTART, run this cell**

In [ ]:
import os
import sys
import time
from pathlib import Path

# Reconstruct paths
username = os.environ.get('USER', 'user')
WORK_DIR = Path(f"/tmp/sf3d_{username}")
VENV_DIR = WORK_DIR / "venv"
SF3D_DIR = WORK_DIR / "stable-fast-3d"
CACHE_DIR = WORK_DIR / "cache"

# Find venv site-packages
for py_ver in ['python3.10', 'python3.11', 'python3.12']:
    venv_site = VENV_DIR / "lib" / py_ver / "site-packages"
    if venv_site.exists():
        sys.path.insert(0, str(venv_site))
        print(f"Using venv: {venv_site}")
        break

sys.path.insert(0, str(SF3D_DIR))
print(f"Using SF3D: {SF3D_DIR}")

# Set HF cache
os.environ['HF_HOME'] = str(CACHE_DIR / 'huggingface')
os.environ['TRANSFORMERS_CACHE'] = str(CACHE_DIR / 'huggingface' / 'transformers')
print(f"HF cache: {os.environ['HF_HOME']}")

# Import and verify PyTorch
import torch
import numpy as np
from PIL import Image

print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️  WARNING: CUDA not available")

# Import SF3D
print("\nImporting SF3D...")
try:
    from sf3d.system import SF3D
    print("✅ SF3D imported successfully")
except ImportError as e:
    print(f"❌ SF3D import failed: {e}")
    print("\nMissing dependency. Check Cell 1 output for errors.")
    raise

# Load model
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\nLoading SF3D model on {device}...")
print("First time: downloads ~2GB, takes 30-60s")

start = time.time()
try:
    model = SF3D.from_pretrained(
        "stabilityai/stable-fast-3d",
        config_name="config.yaml",
        weight_name="model.safetensors",
    )
    model = model.to(device)
    model.eval()
    
    print(f"\n✅ SF3D loaded in {time.time() - start:.1f}s")
    print(f"   Model: stabilityai/stable-fast-3d")
    print(f"   Device: {device}")
except Exception as e:
    print(f"\n❌ Model load failed: {e}")
    import traceback
    traceback.print_exc()
    raise

# Output directory
output_dir = WORK_DIR / "outputs"
output_dir.mkdir(exist_ok=True)
print(f"\nOutput: {output_dir}")
print("\n✅ Ready to generate meshes!")

## Cell 3: Define API

In [ ]:
import io
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import FileResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

app = FastAPI(
    title="SF3D API",
    description="Stable Fast 3D mesh generation",
    version="5.0.0"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {
        "message": "SF3D API (Complete Installation)",
        "status": "running",
        "device": device,
        "model": "stabilityai/stable-fast-3d"
    }

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "device": device,
        "cuda": torch.cuda.is_available(),
        "model_loaded": model is not None
    }

@app.post("/generate")
async def generate(
    file: UploadFile = File(...),
    texture_resolution: int = Form(1024),
    remesh_option: str = Form("none"),
    foreground_ratio: float = Form(0.85)
):
    try:
        data = await file.read()
        image = Image.open(io.BytesIO(data))
        
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"\n[{time.strftime('%H:%M:%S')}] {image.size}")
        
        start = time.time()
        with torch.no_grad():
            output = model.run(
                image,
                bake_resolution=texture_resolution,
                remesh=remesh_option if remesh_option != 'none' else None,
                vertex_count=-1,
            )
        
        gen_time = time.time() - start
        print(f"[{time.strftime('%H:%M:%S')}] {gen_time:.2f}s")
        
        ts = int(time.time() * 1000)
        out_path = output_dir / f"mesh_{ts}.glb"
        
        mesh = output['mesh'] if isinstance(output, dict) else output
        mesh.export(str(out_path))
        
        size = out_path.stat().st_size
        print(f"[{time.strftime('%H:%M:%S')}] {size/1024:.1f}KB\n")
        
        return FileResponse(
            path=out_path,
            media_type="model/gltf-binary",
            filename=f"mesh_{ts}.glb",
            headers={
                "X-Generation-Time": str(gen_time),
                "X-File-Size": str(size)
            }
        )
    except Exception as e:
        print(f"❌ {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(500, str(e))

print("✅ API ready")

## Cell 4: Start Server

In [ ]:
PORT = 8765

print("="*70)
print("🚀 SF3D API Server")
print("="*70)
print(f"URL: http://itp-ml.itp.tsoa.nyu.edu:{PORT}/")
print(f"Device: {device}")
print("="*70)
print(f"\nTest: python tests/sf3d_api_client.py <image> --server http://itp-ml.itp.tsoa.nyu.edu:{PORT}")
print("\n⚠️  KEEP RUNNING\n")

uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")